In [33]:
import pandas as pd 
import numpy as np #maybe delete
import re
from tqdm import tqdm
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [34]:
def getFirstName(df, nameColumn): #nameColumn as string
    firstName = []
    for n in df[nameColumn]:
        nameSplit = re.split(' ', n)
        firstName.append(nameSplit[0])
    return firstName

In [4]:
def assignGender(df):
    index = 0
    nameDict = {}
    genderEstimation = pd.read_csv("Files/wgnd_2_0_name-gender-code.csv") #genderEstimation file
    genderEstimation['Name'] = genderEstimation.name.str.replace(r'"|\'', '', regex=True).str.lower() #remove " and ' and make to lower case
    
    firstName = getFirstName(df, 'Name') #find first names
    df['FirstName'] = firstName #add first name to df
    df['first_name'] = df.FirstName.str.lower() #first name, but lower case
  
    usNames = genderEstimation[genderEstimation.code == 'US'] #some names can't not be found, but can with another code
    otherNames = genderEstimation[~genderEstimation.Name.isin(usNames.Name)] #names not in US

    uniqueNamesUS = df[df.first_name.isin(usNames.Name)].first_name.unique() #df with names from presenters, that is also in genderEstimation
    uniqueNamesOther = df[df.first_name.isin(otherNames.Name)].first_name.unique()
    
    #uniqueNamesUS = us_presenters[us_presenters.first_name.isin(usNames.Name)]['first_name'].unique()
    #uniqueNamesOther = other_presenters[other_presenters.first_name.isin(otherNames.Name)]['first_name'].unique()
    for n in tqdm(uniqueNamesUS): #loop through us names that is also in the presenter df
        nameTable = usNames[usNames.Name == n] #table with the name 
        genderIndex = nameTable.wgt.argmax() #index of gender with highest score/probability
        gender = nameTable.iloc[genderIndex].gender #gender that is most likely
        nameDict[index] = {'first_name': n, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    for m in tqdm(uniqueNamesOther): #loop through other names that is also in the presenter df
        nameTable = otherNames[otherNames.Name == m].groupby('gender').mean() #table with the name - mean of wgt
        genderIndex = nameTable.wgt.argmax() #index of gender with higest average probability
        gender = nameTable.iloc[genderIndex].name
        nameDict[index] = {'first_name': m, 'Gender': gender} #assign gender to the name in a dict
        index += 1
    
    return nameDict

#### Add gender

In [38]:
authors = pd.read_pickle("Files/Dataframes/apiAuthors_realNames.pkl") 

In [6]:
genderDict = assignGender(authors)

100%|█████████████████████████████████████| 10098/10098 [25:42<00:00,  6.55it/s]


In [9]:
#dataframe witht the gender and name
genderAssign = pd.DataFrame.from_dict(genderDict, orient = 'index')

Now we are merging genderAssign and df_presenters. This will automatically add the gender to names in the presenters df.

In [18]:
authors['first_name'] = authors.FirstName.str.lower()

In [19]:
#merge authors df with gender df
authors = authors.merge(genderAssign, how='left', on='first_name').drop(columns=['first_name'])

In [23]:
#only keep authors assigned to either F or M (and not NaN or ?)
authors_wGender = authors[(authors.Gender == 'F') | (authors.Gender == 'M')]

In [25]:
#either F or M assigned
#nAuthors_genderAssigned = len(allAuthors[(allAuthors.Gender == 'F') | (allAuthors.Gender == 'M')]) 
#nPublishedAuthors_gendderAssigned = len(publishedAuthors[(publishedAuthors.Gender == 'F') | (publishedAuthors.Gender == 'M')])

In [26]:
authors_wGender.to_pickle("Files/Dataframes/authors_wGender.pkl") 

In [28]:
print("Number of authors with assigned gender:", len(authors_wGender))

Number of authors with assigned gender: 1467456


In [32]:
len(authors_wGender.Name.unique())

157893